In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, LayerNormalization, Dropout
import pickle
import numpy as np
import json

In [11]:
# Building AlexNet

class AlexNet():
    
    def __init__(self):
        self.model = Sequential()
        self.model.add(Conv2D(96, input_shape=(224, 224, 3), kernel_size=11, activation="relu"))
        self.model.add(LayerNormalization())
        self.model.add(MaxPool2D((3,3)))
        self.model.add(Conv2D(256, kernel_size=5, activation="relu"))
        self.model.add(LayerNormalization())
        self.model.add(MaxPool2D((3,3)))
        self.model.add(Conv2D(384, kernel_size=3, activation="relu"))
        self.model.add(LayerNormalization())
        self.model.add(MaxPool2D((3,3)))
        self.model.add(Conv2D(256, kernel_size=3, activation="relu"))
        self.model.add(LayerNormalization())
        self.model.add(MaxPool2D((3,3)))
        self.model.add(Flatten())
        self.model.add(Dense(4096, activation="leaky_relu"))
        self.model.add(Dropout(0.6))
        self.model.add(Dense(4096, activation="linear"))
        self.model.add(Dropout(0.6))
        self.model.add(Dense(58))
        self.model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.00005), loss="mean_squared_error", metrics="accuracy")

In [12]:
model = AlexNet()
#model.model.summary()

In [20]:
# Create custom dataset

with open('ground_truth', 'rb') as file:
    DATA = pickle.load(file)
    
file_paths = ['FLIC/images/' + idx[1] for idx in DATA]
joint_keyframes = [idx[0] for idx in DATA]

ds_train = tf.data.Dataset.from_tensor_slices((file_paths, joint_keyframes))

def read_image(image_file, joint_keyframes):
    image = tf.io.read_file(image_file)
    image = tf.image.decode_image(image, channels=3, dtype=tf.float32)
    image.set_shape([640, 640, 3])
    image = tf.image.resize(image, [224, 224])
    return image, joint_keyframes

def image_resize(image_file, joint_keyframes):
    image = tf.io.read_file(image_file)
    print(image)
    image =  tf.image.decode_jpeg(image, channels=3, dtype=tf.float32) 
    image_file = tf.image.resize(image, [224,224,3])
    
    return image_file, joint_keyframes


#ds_train = ds_train.map(read_image).batch(4)

In [3]:
history = model.model.fit(ds_train, epochs=100)

In [2]:
import cv2
import matplotlib.pyplot as plt
image = plt.imread('FLIC/images/schindlers-list-00062271.jpg')
plt.imshow(image)
from skimage.transform import resize
resize_image = resize(image, (1, 224,224,3))
y_pred = model.model.predict(resize_image)

import cv2
from matplotlib import pyplot as plt

import time

start = time.time()

fig, ax = plt.subplots(figsize=(10,10))

for i in range(29):
    if y_pred[0][i] > 0:
        ax.scatter(y_pred[0][i], y_pred[0][i+28], color='red')

#ax.imshow(image)

end = time.time()

In [36]:
y_pred

array([[237.35292   , 253.46576   , 241.62775   , 160.2268    ,
        150.50766   , 156.51288   , 227.42314   ,  -0.4979312 ,
         -0.49791232, 165.01013   ,  -0.4965788 ,  -0.49722263,
        197.15715   , 183.89853   ,  -0.49682817,  -0.49558705,
        190.07771   ,  -0.49737823,  -0.49585268,  -0.49794576,
         -0.49677503,  -0.4956996 ,  -0.49538335,  -0.4955399 ,
         -0.49854243,  -0.49781406,  -0.4967903 ,  -0.49741608,
         -0.4973297 ,  90.7979    , 142.84976   , 171.74248   ,
         92.461655  , 144.22765   , 165.51443   , 173.83868   ,
         -0.4970816 ,  -0.49524117, 173.33733   ,  -0.49694332,
         -0.49820414,  54.261497  ,  54.49266   ,  -0.49734214,
         -0.4972309 ,  62.73138   ,  -0.49822092,  -0.49472702,
         -0.4957203 ,  -0.4974459 ,  -0.49701536,  -0.49440193,
         -0.49524322,  -0.495572  ,  -0.49656123,  -0.4987959 ,
         -0.49706984,  -0.49739608]], dtype=float32)